In [54]:
#importing libraries
import os
import numpy as np
import cv2
import os
import shutil
import random
import tensorflow as tf
import imgaug.augmenters as iaa
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model 
from tensorflow.keras.optimizers import Adam


In [55]:
# Setting Dataset Path
images_base_directory = "../Captcha_solver/final_images/"
# images_base_directory = "./my_generated_images"
# images_base_directory = "./final_images"
# Function to get and shuffle image files
def get_and_shuffle_image_files(directory):
    image_files = os.listdir(directory)
    random.shuffle(image_files)
    return image_files



In [56]:
# need to get images names because ismein labels hain

def load_data(image_files):
    images = []
    labels = []
    for file in image_files:
        img = load_img(images_base_directory + "/" +  file, color_mode='grayscale')
        img = img_to_array(img) / 255.0
        label = os.path.basename(file).split('.')[0]
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)

def get_image_files(directory):
    return os.listdir(directory)

image_files = get_and_shuffle_image_files(images_base_directory)
image_files = get_image_files(images_base_directory)    


In [57]:
display(load_data(image_files[0:5])[0].shape)

(5, 80, 190, 1)

### processing labels and creating categorical label

In [58]:
# only loading 80% of data for training
images, labels = load_data(image_files)

# Preprocess the labels

def preprocess_labels(labels, num_characters, num_classes):
    processed_labels = []
    for label in labels:
        # converting individual to int because kaam aayenge
        label = [int(char) for char in label]
        # Convert each integer to categorical
        categorical_label = np.zeros((num_characters, num_classes))
        for i, char in enumerate(label):
            categorical_label[i, char] = 1
        processed_labels.append(categorical_label)
    return np.array(processed_labels)



num_characters = 6  # no of chars
num_classes = 10  # Digits 0-9
labels = preprocess_labels(labels, num_characters, num_classes)


In [59]:


def create_model(input_shape, num_classes, num_characters):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(num_characters * num_classes, activation='softmax'))
    model.add(Reshape((num_characters, num_classes)))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = images[0].shape
model = create_model(input_shape, num_classes, num_characters)

model.fit(images, labels, epochs=30, batch_size=32, validation_split=0.2)
print(input_shape)

Epoch 1/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 87s 598ms/step - accuracy: 0.1395 - loss: 2.1902 - val_accuracy: 0.2744 - val_loss: 2.0215
Epoch 2/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 85s 593ms/step - accuracy: 0.6478 - loss: 0.9707 - val_accuracy: 0.2913 - val_loss: 1.8650
Epoch 3/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 83s 582ms/step - accuracy: 0.8167 - loss: 0.5061 - val_accuracy: 0.2995 - val_loss: 1.8601
Epoch 4/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 84s 589ms/step - accuracy: 0.8662 - loss: 0.3867 - val_accuracy: 0.3202 - val_loss: 1.8581
Epoch 5/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 86s 602ms/step - accuracy: 0.8790 - loss: 0.3453 - val_accuracy: 0.3533 - val_loss: 1.7694
Epoch 6/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 82s 572ms/step - accuracy: 0.8884 - loss: 0.3195 - val_accuracy: 0.3703 - val_loss: 1.7525
Epoch 7/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 82s 576ms/step - accuracy: 0.8944 - loss: 0.2987 - val_accuracy: 0.4137 - val_loss: 1.6912
Epoch 8/30
143/143 ━━━━━━━━━━━━━━━━━━━━ 82s 572ms/step - accuracy: 0.9128 - loss: 0

In [61]:


# Function to resize and normalize an image
def preprocess_image(image_file, target_size):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)  # Load the image in grayscale
    if img is None:
        raise ValueError(f"Image {image_file} not found.")
    img = cv2.resize(img, target_size) 
    img = img / 255.0  
    # adding a channel dimension to the last 
    img = np.expand_dims(img, axis=-1) 
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to predict the CAPTCHA
def predict_captcha(model, preprocessed_img):
    prediction = model.predict(preprocessed_img)
    print(prediction[0])
    predicted_label = ''.join([str(np.argmax(char)) for char in prediction[0]])
    return predicted_label

# Predict a new CAPTCHA image
image_file = images_base_directory + '/' + "116840.png"
preprocessed_img = preprocess_image(image_file, (190, 80))  # Resize to 190x80
captcha = predict_captcha(model, preprocessed_img)
print(f'Predicted CAPTCHA: {captcha}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
[[1.2712940e-13 8.8602197e-01 2.2274328e-08 1.5185835e-08 1.0991174e-11
  4.3842711e-08 2.8127209e-09 1.0896701e-07 5.4500284e-09 7.8799488e-08]
 [7.3605712e-08 8.4312029e-02 1.3376275e-11 4.4702356e-10 6.6629504e-08
  1.5768067e-09 1.4245451e-09 9.3182146e-09 2.0670605e-08 1.2598446e-10]
 [1.3622474e-07 1.8033555e-08 4.1180188e-09 9.5192924e-08 8.3348982e-07
  4.1913918e-08 1.2254922e-02 4.6327076e-09 6.7923412e-07 3.3107330e-07]
 [6.5852657e-07 2.3315842e-08 6.0949624e-08 1.5091697e-05 5.8260905e-08
  1.3695558e-06 1.2245989e-05 1.4899529e-08 6.6617446e-04 4.9733680e-06]
 [9.2611799e-07 4.8204252e-05 8.4520734e-06 9.3068923e-07 2.1465896e-03
  5.7105458e-09 3.4053394e-06 3.4801055e-06 8.1223422e-10 2.2063136e-07]
 [1.4461931e-02 9.2453696e-08 9.8675457e-08 4.0152281e-07 1.9132843e-07
  2.1545045e-08 9.6897350e-07 9.6562438e-08 7.3686027e-07 3.1091378e-05]]
Predicted CAPTCHA: 116840


In [63]:
model.save("model.h5")